In [11]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define the Graph Convolutional Layer
class GraphConvLayer(layers.Layer):
    def __init__(self, output_dim):
        super(GraphConvLayer, self).__init__()
        self.output_dim = output_dim

    def build(self, input_shape):
        self.num_nodes = input_shape[1][1]
        self.num_features = input_shape[1][2]
        self.kernel = self.add_weight("kernel", shape=(self.num_features, self.output_dim))

    def call(self, inputs):
        adjacency_matrix, node_features = inputs
        adjacency_matrix = tf.cast(adjacency_matrix, tf.float32)
        node_features = tf.cast(node_features, tf.float32)
        output = tf.matmul(tf.matmul(adjacency_matrix, node_features), self.kernel)
        return output

# Create a simple graph convolutional model
def create_gcn_model(num_nodes, num_features, num_classes):
    adjacency_matrix = keras.Input(shape=(num_nodes, num_nodes))
    node_features = keras.Input(shape=(num_nodes, num_features))

    hidden = GraphConvLayer(64)([adjacency_matrix, node_features])
    hidden = layers.ReLU()(hidden)
    output = GraphConvLayer(num_classes)([adjacency_matrix, hidden])
    output = layers.Softmax()(output)

    model = keras.Model(inputs=[adjacency_matrix, node_features], outputs=output)
    return model

# Generate random graph data for demonstration
num_nodes = 10
num_features = 16
num_classes = 2

adjacency_matrix = np.random.rand(1, num_nodes, num_nodes)
node_features = np.random.rand(1, num_nodes, num_features)

# Normalize adjacency matrix
adjacency_matrix = adjacency_matrix / np.sum(adjacency_matrix, axis=2, keepdims=True)

# Create the GCN model
model = create_gcn_model(num_nodes, num_features, num_classes)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train the model on random data
labels = np.random.randint(num_classes, size=(1, num_nodes))
model.fit([adjacency_matrix, node_features], labels, epochs=10, batch_size=1)

# Make predictions on new data
new_adjacency_matrix = np.random.rand(1, num_nodes, num_nodes)
new_node_features = np.random.rand(1, num_nodes, num_features)
predictions = model.predict([new_adjacency_matrix, new_node_features])
print(predictions)


Epoch 1/10
1/1 [==============================] - 1s 976ms/step - loss: 0.7349 - accuracy: 0.3000
Epoch 2/10
1/1 [==============================] - 0s 15ms/step - loss: 0.7192 - accuracy: 0.3000
Epoch 3/10
1/1 [==============================] - 0s 12ms/step - loss: 0.7048 - accuracy: 0.3000
Epoch 4/10
1/1 [==============================] - 0s 12ms/step - loss: 0.6915 - accuracy: 0.7000
Epoch 5/10
1/1 [==============================] - 0s 19ms/step - loss: 0.6793 - accuracy: 0.7000
Epoch 6/10
1/1 [==============================] - 0s 18ms/step - loss: 0.6682 - accuracy: 0.7000
Epoch 7/10
1/1 [==============================] - 0s 14ms/step - loss: 0.6581 - accuracy: 0.7000
Epoch 8/10
1/1 [==============================] - 0s 15ms/step - loss: 0.6492 - accuracy: 0.7000
Epoch 9/10
1/1 [==============================] - 0s 14ms/step - loss: 0.6413 - accuracy: 0.7000
Epoch 10/10
1/1 [==============================] - 0s 106ms/step
[[[1.3386017e-04 9.9986625e-01]
  [9.3609601e-04 9.9906391e-0